# Realtime EEG Visualization Tool

In [1]:
from pylsl import StreamInlet, resolve_stream

import sys
import os
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
import functools
import numpy as np
import random as rd
import matplotlib
matplotlib.use("Qt5Agg")
from matplotlib.figure import Figure
from matplotlib.animation import TimedAnimation
from matplotlib.lines import Line2D
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import time
import threading

## Setup LSL Stream

In [2]:
# Set up LSL stream
print("Initialize LSL stream")
lsl_streams = resolve_stream('type', 'EEG')

# create inlet to read from stream
if (len(lsl_streams) < 1):
    error()
lsl_inlet = StreamInlet(lsl_streams[0])

# Get channel names
ch_names = []
curr_ch_child = lsl_inlet.info().desc().child('channels').first_child()
curr_ch_label = curr_ch_child.child_value('label')
while curr_ch_label != '':
    curr_ch_label = curr_ch_child.child_value('label')
    if curr_ch_label != '': ch_names.append(curr_ch_label)
    curr_ch_child = curr_ch_child.next_sibling()
print(ch_names)

# Get sampling frequency
sfreq = lsl_inlet.info().nominal_srate()
print(sfreq)

# Set window size
window_size = 10 # seconds

Initialize LSL stream
['C3', 'C4', 'Cz', 'FPz', 'POz', 'CPz', 'O1', 'O2']
256.0


## Setup Qt Window

In [3]:
class VisualizerMainWindow(QMainWindow):
    def __init__(self):
        super(VisualizerMainWindow, self).__init__()
        
        # Setup window
        self.setGeometry(300, 300, 1200, 600)
        self.setWindowTitle("Realtime EEG Visualization Tool")
        
        # Setup main frame and layout
        self.mainFrame = QFrame(self)
        self.mainFrame.setStyleSheet("QWidget { background-color: %s }" % QColor(180,180,180,255).name())
        self.mainLayout = QGridLayout()
        self.mainLayout.setAlignment(Qt.AlignCenter)
        self.mainFrame.setLayout(self.mainLayout)
        self.setCentralWidget(self.mainFrame)
        
        # Setup plot layout
        self.plotLayout = QGridLayout()
        self.plotLayout.setAlignment(Qt.AlignCenter)
        self.mainLayout.addLayout(self.plotLayout, *(0,0))
        
        # Add channel label
        self.channelLabel = QLabel(self)
        self.channelLabel.setText("Channel:")
        self.plotLayout.addWidget(self.channelLabel, *(0,1))
        
        # Add channel combo box
        self.channelCombo = QComboBox(self)
        self.channelCombo.setStyleSheet("QComboBox { background-color: %s}" % QColor(255,255,255,255).name())
        self.channelCombo.addItem("None")
        for ch in ch_names:
            self.channelCombo.addItem(ch)
        self.channelCombo.setCurrentIndex(0)
        self.plotLayout.addWidget(self.channelCombo, *(0,2))
        self.channelCombo.currentIndexChanged.connect(self.changeChannel)
        
        # Add display type label
        self.displayTypeLabel = QLabel(self)
        self.displayTypeLabel.setText("Display Type:")
        self.plotLayout.addWidget(self.displayTypeLabel, *(1,1))
        
        # Add display type combo box
        self.displayTypeCombo = QComboBox(self)
        self.displayTypeCombo.setStyleSheet("QComboBox { background-color: %s}" % QColor(255,255,255,255).name())
        self.displayTypeCombo.addItem("Time")
        self.displayTypeCombo.addItem("Frequency")
        self.displayTypeCombo.addItem("Time-Frequency")
        self.displayTypeCombo.setCurrentIndex(0)
        self.plotLayout.addWidget(self.displayTypeCombo, *(1,2))
        self.displayTypeCombo.currentIndexChanged.connect(self.changeDisplayType)
        
        # Add display plot
        self.displayPlot = DisplayCanvas()
        self.plotLayout.addWidget(self.displayPlot, *(2,0,1,4))
        
        # Add zoom in button
        self.zoomInBtn = QPushButton(text = 'Zoom in')
        self.zoomInBtn.setStyleSheet("QPushButton { background-color: %s}" % QColor(255,255,255,255).name())
        self.zoomInBtn.setFixedSize(100, 50)
        self.zoomInBtn.clicked.connect(self.zoomInBtnAction)
        self.plotLayout.addWidget(self.zoomInBtn, *(3,1))
        
        # Add zoom out button
        self.zoomOutBtn = QPushButton(text = 'Zoom out')
        self.zoomOutBtn.setStyleSheet("QPushButton { background-color: %s}" % QColor(255,255,255,255).name())
        self.zoomOutBtn.setFixedSize(100, 50)
        self.zoomOutBtn.clicked.connect(self.zoomOutBtnAction)
        self.plotLayout.addWidget(self.zoomOutBtn, *(3,2))
        
        # Add callbackfunc to add data
        myDataLoop = threading.Thread(name = 'myDataLoop',
                                      target = dataSendLoop,
                                      daemon = True,
                                      args = (self.addData_callbackFunc,))
        myDataLoop.start()
        
        self.show()
        return

    def zoomInBtnAction(self):
        self.displayPlot.zoomIn(100)
        return
    
    def zoomOutBtnAction(self):
        self.displayPlot.zoomOut(100)
        return

    def addData_callbackFunc(self, value):
        self.displayPlot.addData(value)
        return
    
    def changeChannel(self):
        self.displayPlot.changeChannel(self.channelCombo.currentIndex())
        
    def changeDisplayType(self):
        self.displayPlot.changeDisplayType(self.displayTypeCombo.currentText())

In [4]:
class DisplayCanvas(FigureCanvas, TimedAnimation):
    def __init__(self):
        self.addedData = []
        
        # display parameters
        self.sfreq = int(sfreq) # Hz
        self.windowSize = window_size # seconds
        self.xlim = self.sfreq * self.windowSize
        self.n = np.linspace(0, self.xlim - 1, self.xlim) / self.sfreq
        self.y = self.n * 0.0
        self.currentChannel = 0
        self.displayType = "Time"
        
        # setup window
        self.fig = Figure(figsize=(10,5), dpi=100)
        self.ax1 = self.fig.add_subplot(111)
        
        # setup plot
        self.ax1.set_xlabel('Time')
        self.ax1.set_ylabel('Potential (microVolts)')
        self.ax1.set_xlim(0, self.n[-1])
        self.ax1.set_ylim(-1000, 1000)
        
        self.line1 = Line2D([], [], color='blue')
        self.line1_tail = Line2D([], [], color='red', linewidth=2)
        self.line1_head = Line2D([], [], color='red', marker='o', markeredgecolor='r')
        self.ax1.add_line(self.line1)
        self.ax1.add_line(self.line1_tail)
        self.ax1.add_line(self.line1_head)
        
        FigureCanvas.__init__(self, self.fig)
        TimedAnimation.__init__(self, self.fig, interval = 50, blit = True)
        return

    def new_frame_seq(self):
        return iter(range(self.n.size))

    def _init_draw(self):
        lines = [self.line1, self.line1_tail, self.line1_head]
        for l in lines:
            l.set_data([], [])
        return

    def addData(self, value):
        self.addedData.append(value)
        return

    def zoomIn(self, value):
        bottom = self.ax1.get_ylim()[0]
        top = self.ax1.get_ylim()[1]
        bottom += value
        top -= value
        self.ax1.set_ylim(bottom,top)
        self.draw()
        return
    
    def zoomOut(self, value):
        bottom = self.ax1.get_ylim()[0]
        top = self.ax1.get_ylim()[1]
        bottom -= value
        top += value
        self.ax1.set_ylim(bottom,top)
        self.draw()
        return
    
    def changeChannel(self, channelIndex):
        self.currentChannel = channelIndex
        return
    
    def changeDisplayType(self, displayType):
        self.displayType = displayType
        if self.displayType == "Time":
            # TODO
            aaa = 1
        elif self.displayType == "Frequency":
            #TODO
            aaa = 1
        elif self.displayType == "Time-Frequency":
            #TODO
            aaa = 1
        return

    def _step(self, *args):
        # Extends the _step() method for the TimedAnimation class.
        try:
            TimedAnimation._step(self, *args)
        except Exception as e:
            self.abc += 1
            print(str(self.abc))
            TimedAnimation._stop(self)
            pass
        return

    def _draw_frame(self, framedata):
        margin = 2
        
        # Update data buffer
        while(len(self.addedData) > 0):
            self.y = np.roll(self.y, -1)
            if self.currentChannel == 0:
                self.y[-1] = 0
            else:
                self.y[-1] = self.addedData[0][self.currentChannel-1]
            del(self.addedData[0])
        
        # Update plot
        if self.displayType == "Time":
            self.line1.set_data(self.n[ 0 : self.n.size - margin ], self.y[ 0 : self.n.size - margin ])
            self.line1_tail.set_data(np.append(self.n[-10:-1 - margin], self.n[-1 - margin]), np.append(self.y[-10:-1 - margin], self.y[-1 - margin]))
            self.line1_head.set_data(self.n[-1 - margin], self.y[-1 - margin])
        elif self.displayType == "Frequency":
            self.line1.set_data([], [])
            self.line1_tail.set_data([], [])
            self.line1_head.set_data([], [])
        elif self.displayType == "Time-Frequency":
            self.line1.set_data([], [])
            self.line1_tail.set_data([], [])
            self.line1_head.set_data([], [])
        
        self._drawn_artists = [self.line1, self.line1_tail, self.line1_head]
        return

In [5]:
# Setup signal slot mechanism
class Communicate(QObject):
    data_signal = pyqtSignal(list)

In [6]:
def dataSendLoop(addData_callbackFunc):
    # Setup the signal-slot mechanism.
    mySrc = Communicate()
    mySrc.data_signal.connect(addData_callbackFunc)

    while(True):
        sample, timestamp = lsl_inlet.pull_sample()
        mySrc.data_signal.emit(sample)
        time.sleep(0.01)

In [7]:
if __name__== '__main__':
    app = QApplication(sys.argv)
    QApplication.setStyle(QStyleFactory.create('Plastique'))
    visualizer = VisualizerMainWindow()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\Jason\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
